TO DO:
- Make generic functions for tasks for modularity
- CRON jobs for all timeframes
- One function to update all csvs
- Plug n play for indicators
- Add limit to queries
- Include Active/Inactive column in CSV for active/inactive coins
- Update_CSV_to_Latest should contain active coins from Exchanges and From CSV. Check if a (coin,exchange) tuple is active(Check if it is present on exchange using ccxt library) . If it is active   get latest data for it if already present in CSV,if not in CSV get all data. If coin is not active on exchange , we will put a Active/Inactive status in CSV accordingly. All functions will have   to be modified to run code only for active coin-exchange combinations.
- Date Format will be '%d-%m-%Y %H:%M:%S' . This is giving me a lot of problems especially while reading data. When I don't put :%S it tells me dataframe has second and sometimes when :%S is         there, it tells me no second value in dataframe.
- Analytics Value Accuracy. Some parameter in Jupyter.
- Have to fetch Coins based on Parameters. Example - Fetch all active coins-exchange combinations where RSI>0 and RSI<=30. Fetch all active coins-exchange combinations where closing price is         between LOWERBAND and MIDDLEBAND. Get me intersection(common coins) of these 2 list. Now the coin from the intersection list which probably has the lowest volume can increase in price faster       then the others(Little increase in Volume will result in Big increase in Price)
- For each active coin-exchange combination I want to check the change in Value of different Technical Indicators of 2 consecutive periods in time. Example - I want to know if for a particular       coin RSI=a on period x and RSI>a on period x+1. I want to know whenever MACD and MACD_SIGNAL cross each other(On period x MACD=a and MACD_SIGNAL=b where a<=b and on period x+1 MACD=a and           MACD_SIGNAL=b where a>b. MACD_HISTOGRAM same like RSI want to know when it is 'a' on period x and 'a++' on period x+1).
- Convert 1D timeframe to 3D/1Week/etc. Convert 1H timeframe to 4H/6H/etc.
- Batch Processing of downloading new coins data

In [36]:
# Imports
import os
import time
import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser
import ccxt
import numpy as np
import pandas as pd
from importlib import reload
from collections import defaultdict
import sys
import inspect
import pyti
from apscheduler.schedulers.background import BackgroundScheduler

In [37]:
importlib.reload(logging)
LOGGING_FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(format=LOGGING_FORMAT, level=logging.
                    INFO)


In [38]:
cryptocompare_wrapper = 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\notebooks\\cryptocompare_wrapper.py'
cryptocompare_wrapper

'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\notebooks\\cryptocompare_wrapper.py'

In [39]:
import cryptocompare_wrapper as ccw
reload(ccw)

<module 'cryptocompare_wrapper' from 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\notebooks\\cryptocompare_wrapper.py'>

In [40]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'
NPERIODS = 100
TIMEFRAME = 'Day'
datetimeStringformat_to_csv = "%d-%m-%Y %H:%M"
data_directory = 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\data\\'

In [41]:
# Defaults
CURR = 'BTC'
EXCHANGE = 'CCCAGG'
COIN = 'ETH'
COIN_LIST = ['BTC', 'ETH', 'XRP']
EXCHANGES = ['Bittrex','Binance','Kucoin','HuobiPro','Cryptopia','IDEX']
EXCHANGES = ['Binance']
bittrex_exchange = ccxt.bittrex()
binance_exchange = ccxt.binance()
kucoin_exchange = ccxt.kucoin()
huobiPro_exchange = ccxt.huobipro()
cryptopia_exchange = ccxt.cryptopia()
bitmex_exchange = ccxt.bitmex()
#print(bittrex_exchange.fetchCurrencies())
list_of_exchanges = [bittrex_exchange,binance_exchange,kucoin_exchange,huobiPro_exchange,
                     cryptopia_exchange]
#list_of_exchanges = [bitmex_exchange]

In [42]:
# For every exchange, fetch it's markets. Then depending on the JSON returned, prepare a list of coins for which historical data has to be downloaded.
def setupExchanges(list_of_exchanges):
    done = False
    i=0
    #df_markets = pd.DataFrame(markets)
    #bittrex_market = bittrex_exchange.fetchMarkets()
    #binance_market = binance_exchange.fetchMarkets()
    #kucoin_market = kucoin_exchange.fetchMarkets()
    #list_of_markets = [#bittrex_market,
                       #binance_market
     #                  kucoin_market #For kucoin the fetchMarkets function returns different dictionary keys
      #                  ]

    var_quote = ""
    coin_exchange_combination = {}
    for exchange in list_of_exchanges:
        coins_list = set()
        #if exchange.name == 'Cryptopia' or exchange.name == 'Bittrex' or exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
            #continue #exchange.name == 'Binance' or 
        markets = exchange.fetchMarkets()
        for row in markets:
            if exchange.name == 'Huobi Pro' or exchange.name == 'Cryptopia':
                if row['base'] not in coins_list:
                        coins_list.add(row['base'])
                #continue
            if  'active' in row and row['active'] == True :
                #print(exchange.name,row)
                #sys.exit("Te")
                if exchange.name == 'Bittrex' or exchange.name == 'Binance'  :
                    var_quote = "quoteId"
                elif   exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
                    var_quote = "quote"
                #print(var_quote)
                if var_quote in row and row[var_quote] == 'BTC':
                    if row['base'] not in coins_list:
                        coins_list.add(row['base'])
        coin_exchange_combination[exchange.name] = coins_list
    return coin_exchange_combination

In [43]:
def convert_timestamp(timestamp):
    """ Convert timestamp into readable datetime """
    try:
        return datetime.datetime.fromtimestamp(int(timestamp)).strftime('%d-%m-%Y %H:%M')
        #return timestamp
    except Exception as e:
        logging.debug(e)
        return None

In [44]:
def update_and_delete_coin_exchange_combination(csv_filename_read,csv_filename_write,timeframe,coin_exchange_combination):
    #For every exchange, download the new coins and delete coins which are delisted from that exchange
    csv_all_coins_full = csv_filename_read
    csv_all_coins_full_new = csv_filename_write
    not_updated = defaultdict(list)
    existing_coin_exchange = []
    # If the csv already exists, find out which coins and exchanges have already been added
    if os.path.isfile(csv_all_coins_full):
        df_csv_all_coins_full = pd.read_csv(csv_all_coins_full,index_col=['exchange','coin'])
        # existing_coin_exchange is a list of tuples (coin, exchange)
        existing_coin_exchange = np.unique(df_csv_all_coins_full.index.values)
        coin_exchange_combination_in_excel = {}
        for a, b in existing_coin_exchange:
            coin_exchange_combination_in_excel.setdefault(a, []).append(b)
    number_of_coins = 0
    coin_exchange_combination_to_delete = {}
    for exchange in coin_exchange_combination :
        if exchange not in coin_exchange_combination_in_excel:
            continue
        coins_list_from_exchange = coin_exchange_combination[exchange]
        coins_list_from_excel = coin_exchange_combination_in_excel[exchange]
        coins_to_download = list(set(coins_list_from_exchange) - set(coins_list_from_excel))
        coins_to_delete = list(set(coins_list_from_excel) - set(coins_list_from_exchange))
        coin_exchange_combination_to_delete[exchange] = coins_to_delete
        print(exchange,coins_to_download)
        for symbol in coins_to_download:
         #For every symbol-exchange combination, if it is present in CSV,don't download historical Data for it.
            try:
                # Can't fetch the same symbol in same symbol rate
                print(symbol,exchange,"In Try Block")
                func = function_period_mapping[timeframe]
                to_curr = 'BTC'
                if symbol == "BTC":
                    to_curr = "USD"
                if exchange == 'IDEX':
                    to_curr = 'ETH'
                if symbol is not to_curr:
                    df_coin_all = func(
                        coin=symbol,
                        to_curr=to_curr,
                        timestamp=time.time(),
                        exchange=exchange
                    )
    
                if df_coin_all.empty:
                    not_updated[exchange].append(symbol)
                    #print(symbol,exchange)
                else:
                    df_coin_all['exchange'] = exchange
                    df_coin_all['coin'] = symbol
                    print("Coin Inserted")
                    df_coin_all = df_coin_all.reset_index()
                    df_coin_all['time'] = df_coin_all.unix_timestamp.apply(convert_timestamp)
                    print("Time column created")
                    df_coin_all = df_coin_all.set_index(['coin','exchange', 'unix_timestamp'])
                    print("Index Set Again")
                    # If csv does not exist, write, else append
                    if not os.path.isfile(csv_all_coins_full_new):
                        print("New File Created")
                        df_coin_all.to_csv(csv_all_coins_full_new, mode='w')
                    else:
                        print("Appended")
                        df_coin_all.to_csv(csv_all_coins_full_new, mode='a', header=False)
                    number_of_coins = number_of_coins +1
    
            except Exception as e:
                logging.error(e)
                # logging.debug("Could not update data for {curr} from {exchange}".format(curr=symbol, exchange=exchange))
                not_updated[exchange].append(symbol)
    
    logging.error("Did not update the following. Try again.\n {not_updated}".format(not_updated=not_updated))
    print(number_of_coins)
    print(coin_exchange_combination_to_delete)
    delete_coins_from_CSV(coin_exchange_combination_to_delete,df_csv_all_coins_full.reset_index(),csv_filename_read)

In [45]:
def delete_coins_from_CSV(coin_exchange_combination_to_delete,df_csv_all_coins_full,csv_filename_read):
    for exchange in coin_exchange_combination_to_delete:
        coins_list_to_delete = coin_exchange_combination_to_delete[exchange]
        for symbol in coins_list_to_delete:
            df_csv_all_coins_full = df_csv_all_coins_full[~((df_csv_all_coins_full['coin'] == symbol) & (df_csv_all_coins_full['exchange'] == exchange))] 
    df_csv_all_coins_full.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename_read)

In [ ]:
coins_list_from_exchange = setupExchanges(list_of_exchanges)
update_and_delete_coin_exchange_combination(data_directory+'all_coins_day_full_1day.csv',data_directory+'all_coins_day_full_1day_new_coins.csv','1dayfull',coins_list_from_exchange)

In [ ]:
#coins_list = ["BTC"]
coins_list = [""]
setupExchanges(list_of_exchanges)
update_and_delete_coin_exchange_combination(data_directory+'all_coins_hour_full_1hour_.csv',data_directory+'all_coins_hour_full_1hour_.csv','1hour',coins_list)

In [46]:
# Maps csv (future data objects) to period granularity
# If we store all data together in a single data source, we'll change this to a function which returns corresponding rows
data_csv_period_mapping = {
    "1day": data_directory+'all_coins_day_full_1day.csv',
    "1hour": data_directory+'all_coins_hour_full_1hour.csv',
    "1min": data_directory+'all_coins_min_full_1min.csv',
    "1daycryptopia":data_directory+'all_coins_day_full_1day_Cryptopia.csv',
    '1daybtcbitfinex':data_directory+'BTC_Bitfinex_day_full_1day.csv',
    '1hourbtcbitfinex':data_directory+'BTC_Bitfinex_hour_full_1hour.csv'
}
frequency_resampling_period_mapping = {
    "day":'D',
    "hour":'H',
    "min":'M'
}
function_period_mapping = {
    '1day': ccw.get_historical_price_day,
    '1hour': ccw.get_historical_price_hour,
    '1min': ccw.get_historical_price_minute,
    '1dayfull' : ccw.get_historical_price_day_full,
    '1daycryptopia' : ccw.get_historical_price_day,
    '1daybtcbitfinex':ccw.get_historical_price_day,
    '1hourbtcbitfinex':ccw.get_historical_price_hour
}

In [47]:
#df_csv = pd.read_csv('all_coins_day_full.csv', index_col=None)
indicator_list = ['unix_timestamp','BBANDS_BANDWIDTH_PERCENT','MONEY_FLOW_INDEX',
                   'STOCH_PERCENT_K_MONEY_FLOW_INDEX','STOCH_PERCENT_D_MONEY_FLOW_INDEX','RSI','RSI_OVER_BOUGHT','RSI_OVER_SOLD',
                   'STOCHRSI_K','STOCHRSI_D','STOCH_PERCENT_K','STOCH_PERCENT_D','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST',
                  'MACD','MACD_SIGNAL','MACD_TEST','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST']   

In [48]:
 # Technical Analysis Settings
EMA_FAST = 10
EMA_SLOW = 20
RSI_PERIOD = 14
RSI_OVER_BOUGHT = 70
RSI_OVER_SOLD = 30
RSI_AVG_PERIOD = 15
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
STOCH_K = 14
STOCH_D = 3
STOCH_OVER_BOUGHT = 70
STOCH_OVER_SOLD = 30
from pyti import bollinger_bands
from pyti import money_flow_index
from pyti import stochastic
from pyti import simple_moving_average
from pyti import stochrsi
from pyti import on_balance_volume

In [49]:
def update_indicator(csv_filename,periods,timeframe,datetimeformat_string):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    df_csv.drop_duplicates(subset=['coin','exchange','unix_timestamp'],inplace=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M:%S'))
    
    for indicator in indicator_list:
        if indicator not in df_csv.columns and indicator not in df_csv.index:
            df_csv[indicator] = np.nan
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data2 = req_data.iloc[-periods:]

            start_date = req_data2.index.get_level_values(2)[0]
            end_date = req_data2.index.get_level_values(2)[req_data2.shape[0]-1]
            req_data2 = req_data[(req_data.index.get_level_values(2) >= start_date) & (req_data.index.get_level_values(2) <= end_date)]
            #print(req_data2)
            np_volumeto = np.array(req_data2.volumeto.values,dtype='f8')
            if len(np_volumeto)<20:
                j = j+1
                print(coin_name,j," Not Updated")
                continue
            req_data2['BBANDS_BANDWIDTH_PERCENT'] = pyti.bollinger_bands.percent_b(req_data2.close.values,20)
            req_data2['MONEY_FLOW_INDEX'] = money_flow_index.money_flow_index(req_data2.close.values, req_data2.high.values, req_data2.low.values, np_volumeto, 14)
            req_data2['STOCH_PERCENT_K_MONEY_FLOW_INDEX'] = pyti.stochastic.percent_k(req_data2.MONEY_FLOW_INDEX.values,14) * 100
            req_data2['STOCH_PERCENT_D_MONEY_FLOW_INDEX'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCH_PERCENT_K_MONEY_FLOW_INDEX.values,3)
            req_data2['RSI'] = talib.RSI(req_data2.close.values, timeperiod=RSI_PERIOD)
            req_data2['RSI_OVER_BOUGHT'] = np.where((req_data2.RSI >= RSI_OVER_BOUGHT) & (req_data2.RSI <= req_data2.RSI.shift(1)),1,0)
            req_data2['RSI_OVER_SOLD'] = np.where((req_data2.RSI <= RSI_OVER_SOLD) & (req_data2.RSI >= req_data2.RSI.shift(1)),1,0)
            req_data2['STOCHRSI_K'] = pyti.stochrsi.stochrsi(req_data2.close.values,14)
            req_data2['STOCHRSI_D'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCHRSI_K.values,3)
            req_data2['STOCH_PERCENT_K'] = pyti.stochastic.percent_k(req_data2.high.values,14) * 100
            req_data2['STOCH_PERCENT_D'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCH_PERCENT_K.values,3)
            req_data2['STOCH_OVER_BOUGHT'] = np.where((req_data2.STOCH_PERCENT_K >= STOCH_OVER_BOUGHT) & (req_data2.STOCH_PERCENT_K <= req_data2.STOCH_PERCENT_K.shift(1)),1,0)
            req_data2['STOCH_OVER_SOLD'] = np.where((req_data2.STOCH_PERCENT_K <= STOCH_OVER_SOLD) & (req_data2.STOCH_PERCENT_K >= req_data2.STOCH_PERCENT_K.shift(1)),1,0)
            req_data2['SMA_FAST'] = talib.SMA(req_data2.close.values,7)
            req_data2['SMA_SLOW'] = talib.SMA(req_data2.close.values,21)
            req_data2['SMA_TEST'] = np.where(req_data2.SMA_FAST>req_data2.SMA_SLOW,1,0)
            req_data2['ON_BALANCE_VOLUME'] = on_balance_volume.on_balance_volume(req_data2.close.values,np_volumeto)
            req_data2['ON_BALANCE_VOLUME_TEST'] = np.where(req_data2.ON_BALANCE_VOLUME>req_data2.ON_BALANCE_VOLUME.shift(1),1,0)
            """
            req_data2['Accumulation_Distribution_Oscillator'] = talib.ADOSC(req_data2.high.values,req_data2.low.values
                                                  ,req_data2.close.values,np_volumeto)
            req_data2['ADOSC_TEST'] = np.where((req_data2.Accumulation_Distribution_Oscillator>req_data2.Accumulation_Distribution_Oscillator.shift(1)) & (req_data2.Accumulation_Distribution_Oscillator>=0) & 
                                               (req_data2.Accumulation_Distribution_Oscillator.shift(1)<=0),1,0)
            """
            
            req_data2['MACD'],req_data2['MACD_SIGNAL'],MACD_HISTOGRAM= talib.MACD(req_data2.close.values,fastperiod=MACD_FAST,slowperiod=MACD_SLOW,signalperiod=MACD_SIGNAL)
            req_data2['MACD_TEST'] = np.where(req_data2.MACD>req_data2.MACD_SIGNAL,1,0)
            
            
            df_csv.update(req_data2)
            i = i+1
            print(coin_name,i)
            #print(df_csv.query('coin == @coin_name').tail(1))
            #sys.exit("Testing")
    df_csv.to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [65]:
update_indicator(data_directory+'all_coins_day_full_3days.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")  

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

XWC 1


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

XDN 2


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

...

GLD 121
SYNX 122
UNB 123
EBST 124
VRM 125
NXS 1  Not Updated
NXS 126
REP 127
REP 128
SHIFT 129
ARDR 130
XZC 131
XZC 132
NEO 133
NEO 134
NEO 135
ZEC 136
ZEC 137
ZCL 138
IOP 139
GOLOS 140
UBQ 141
GBG 142
KMD 143
KMD 144
PIVX 145
PIVX 146
ION 147
LMC 148
SIB 149
QWARK 150
CRW 151
SWT 152
MLN 153
SEQ 154
ARK 155
ARK 156
DYN 157
TKS 158
MUSIC 159
DTB 160
INCNT 161
GBYTE 162
GNT 163
GNT 164
NXC 165
EDG 166
RLC 167
RLC 168
WINGS 169
WINGS 170
GUP 171
GNO 172
LUN 173
LUN 174
HMQ 175
ANT 176
SC 2  Not Updated
SC 177
BAT 178
BAT 179
ZEN 3  Not Updated
ZEN 180
QRL 181
PTOY 182
CFI 183
NMR 184
SNT 185
SNT 186
SNT 187
DCT 188
MCO 189
MCO 190
XEL 191
ADT 192
PAY 193
PAY 194
ADX 195
ADX 196
STORJ 197
STORJ 198
BNB 199
CVC 4  Not Updated
CVC 200
CVC 201
OMG 202
OMG 203
OMG 204
PART 205
QTUM 206
QTUM 207
QTUM 208
BCH 209
BCH 210
BCH 211
GAS 212
GAS 213
HSR 214


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


HSR 215
WTC 216
WTC 217


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\relative_strength_index.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  rsi.append(100 - (100 / (1 + rs)))
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\relative_strength_index.py:47: RuntimeWarning: divide by zero encountered in double_scalars
  rsi.append(100 - (100 / (1 + rs)))
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encounter

LRC 218
LRC 219
YOYOW 220
BQX 221
FUN 222
KNC 223
KNC 224
LINK 225
SNGLS 226
SNM 227
SNM 228
ZRX 229
ZRX 230
ADA 231
ADA 232
EOS 233
EOS 234
MDA 235
MTL 236
SALT 237
SALT 238
SUB 239
SUB 240
MANA 241
MANA 242
MANA 5  Not Updated
DNT 243
DNT 244
TIX 245
AST 246
BTG 247
BTG 248
BTG 249
ENG 250
ENG 251
ENJ 252
ENJ 253
EVX 254
ICN 255
MOD 256
MOD 257
MTH 258
MTH 259
OAX 260
POWR 261
POWR 262
POWR 263
RCN 264
RCN 265
REQ 266
REQ 267
TRX 268
TRX 269
VIB 270
VIB 271
MER 272
AMB 273
AMB 274
ARN 275
ARN 6  Not Updated
BCPT 276
BCPT 277
BCPT 278
DLT 279
NULS 280
NULS 281
VEN 282
VEN 283
QSP 284
QSP 285
BCD 286
BCD 287
CAG 288
GVT 289
GVT 290
HST 291
KCS 292
NEBL 293
NEBL 294
PPT 295
PPT 296
PURA 297
RPX 298
RPX 299
UKG 300
UKG 301
XAS 302
BTS 303
CDT 304
CND 305
DGD 306
FUEL 307
GXS 308
LEND 309
LEND 310
POE 311
TNT 312
WABI 313
AION 314
AION 315
BNTY 316
BRD 317
BRD 7  Not Updated
DBC 318
DENT 8  Not Updated
DENT 319
DRGN 320
EDO 321
ELF 322
ELF 9  Not Updated
ELIX 323
FLIXX 324
GTO 325
GTO 10 

In [64]:
update_indicator(data_directory+'all_coins_day_full_7days.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")  

XWC 1
XDN 2
EMC2 3


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 1  Not Updated
CLOAK 15
DOPE 16
FTC 17
IOC 18
MONA 19
SYS 2  Not Updated
SYS 20
VRC 21
XLM 22
XLM 23
XMG 24
XMY 25
BLK 26
BURST 27
BYC 28
CANN 29
DGB 30
DGB 31
DOGE 32
EFL 33
EXCL 34
LTC 35
LTC 36
LTC 37
NAV 38
NAV 39
NEOS 40
NLG 41
NXT 42
POT 43
PTC 44
SLR 45
SPR 46
TRUST 47
VIA 48
VIA 49
VTC 50
XMR 51
XMR 52
XRP 53
XRP 54
XST 55
XCP 56
BITB 57
GEO 58
FLDC 59
GRC 60
FLO 61
DASH 62
DASH 63
DASH 64
CLAM 65
NBT 66
XEM 3  Not Updated
XEM 67
DMD 68
MUE 69
GAM 70
SPHR 71
OK 72
PPC 73
ERC 74
AEON 75
ETH 76
ETH 77
ETH 78
TX 79
BCY 80
EXP 81
OMNI 82
AMP 83
AUR 84
BNT 85


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


BNT 86
RVR 87
EMC 88
FCT 89
EGC 90
SLS 91
GRS 4  Not Updated
GRS 92
RADS 93
DCR 94
BSD 95
XVG 96
XVG 97
MEME 98
GAME 99
STEEM 100
STEEM 101
2GIVE 102
LSK 103
LSK 104
BRK 105
WAVES 106
WAVES 107
LBC 108
SBD 109
BRX 110
ETC 111
ETC 112
ETC 113
STRAT 114
STRAT 115


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]


COVAL 116
GLD 117
SYNX 118
UNB 119
EBST 120
VRM 121
NXS 5  Not Updated
NXS 122
REP 6  Not Updated
REP 123
SHIFT 124
ARDR 125
XZC 126
XZC 127
NEO 128
NEO 129
NEO 130
ZEC 131
ZEC 132
ZCL 133
IOP 134
GOLOS 135
UBQ 136
GBG 137
KMD 138
KMD 139
PIVX 140
PIVX 141
ION 142
LMC 143
SIB 144
QWARK 145
CRW 146
SWT 147
MLN 148
SEQ 149
ARK 150
ARK 151
DYN 152
TKS 153
MUSIC 154
DTB 155
INCNT 156
GBYTE 157
GNT 7  Not Updated
GNT 158
NXC 159
EDG 160
RLC 161
RLC 162
WINGS 163
WINGS 164
GUP 165
GNO 166
LUN 167
LUN 168
HMQ 169
ANT 170
SC 8  Not Updated
SC 171
BAT 172
BAT 173
ZEN 9  Not Updated
ZEN 174
QRL 175
PTOY 176
CFI 177
NMR 178
SNT 179
SNT 180
SNT 181
DCT 182
MCO 183
MCO 184
XEL 185
ADT 186
PAY 187
PAY 188
ADX 189
ADX 190
STORJ 191
STORJ 192
BNB 193
CVC 10  Not Updated
CVC 194
CVC 195
OMG 196
OMG 197
OMG 198
PART 199
QTUM 200
QTUM 201
QTUM 202
BCH 203
BCH 204
BCH 205
GAS 206
GAS 207
HSR 208
HSR 209
WTC 210
WTC 211
LRC 212
LRC 11  Not Updated
YOYOW 213
BQX 214
FUN 215
KNC 216
KNC 217
LINK 218
SNGLS 21

In [63]:
update_indicator(data_directory+'all_coins_day_full_10days.csv',250,'10day',datetimeStringformat_to_csv)
print("Done")  

XWC 1
XDN 2
EMC2 3


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochrsi.py:16: RuntimeWarning: invalid value encountered in double_scalars
  stochrsi = [100 * ((rsi[idx] - np.min(rsi[idx+1-period:idx+1])) / (np.max(rsi[idx+1-period:idx+1]) - np.min(rsi[idx+1-period:idx+1]))) for idx in range(period-1, len(rsi))]
C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 1  Not Updated
CLOAK 15
DOPE 16
FTC 17
IOC 18
MONA 19
SYS 2  Not Updated
SYS 20
VRC 21
XLM 22
XLM 23
XMG 24
XMY 25
BLK 26
BURST 27
BYC 28
CANN 29
DGB 30
DGB 31
DOGE 32
EFL 33
EXCL 34
LTC 35
LTC 36
LTC 37
NAV 38
NAV 39
NEOS 40
NLG 41
NXT 42
POT 43
PTC 44
SLR 45
SPR 46
TRUST 47
VIA 3  Not Updated
VIA 48
VTC 49
XMR 50
XMR 51
XRP 52
XRP 53
XST 54
XCP 55
BITB 56
GEO 57
FLDC 58
GRC 59
FLO 60
DASH 61
DASH 62
DASH 63
CLAM 64
NBT 65
XEM 4  Not Updated
XEM 66
DMD 67
MUE 68
GAM 69
SPHR 70
OK 71
PPC 72
ERC 73
AEON 74
ETH 75
ETH 76
ETH 77
TX 78
BCY 79
EXP 80
OMNI 81
AMP 82
AUR 83
BNT 84
BNT 85
RVR 86
EMC 87


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


FCT 88
EGC 89
SLS 90
GRS 5  Not Updated
GRS 91
RADS 92
DCR 93
BSD 94
XVG 95
XVG 96
MEME 97
GAME 98
STEEM 6  Not Updated
STEEM 99
2GIVE 100
LSK 101
LSK 102
BRK 103
WAVES 104
WAVES 105
LBC 106
SBD 107
BRX 108
ETC 109
ETC 110
ETC 111
STRAT 112
STRAT 113
COVAL 114
GLD 115
SYNX 116
UNB 117
EBST 118
VRM 119
NXS 7  Not Updated
NXS 120
REP 8  Not Updated
REP 121
SHIFT 122
ARDR 123
XZC 124
XZC 125
NEO 126
NEO 127
NEO 128
ZEC 129
ZEC 130
ZCL 131
IOP 132
GOLOS 133
UBQ 134
GBG 135
KMD 136
KMD 137
PIVX 9  Not Updated
PIVX 138
ION 139
LMC 140
SIB 141
QWARK 142
CRW 143
SWT 144
MLN 145
SEQ 146
ARK 147
ARK 148
DYN 149
TKS 150
MUSIC 151
DTB 152
INCNT 153
GBYTE 154
GNT 10  Not Updated
GNT 155
NXC 156
EDG 157
RLC 11  Not Updated
RLC 158
WINGS 159
WINGS 160
GUP 161
GNO 162
LUN 163
LUN 164
HMQ 165
ANT 166
SC 12  Not Updated
SC 167
BAT 168
BAT 169
ZEN 13  Not Updated
ZEN 170
QRL 171
PTOY 172
CFI 173
NMR 174
SNT 175
SNT 176
SNT 177
DCT 178
MCO 179
MCO 180
XEL 181
ADT 182
PAY 183
PAY 184
ADX 185
ADX 186
STORJ 

In [62]:
update_indicator(data_directory+'all_coins_day_full_14days.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")  

XWC 1
XDN 2
EMC2 3


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 1  Not Updated
CLOAK 15
DOPE 16
FTC 17
IOC 18
MONA 19
SYS 2  Not Updated
SYS 20
VRC 21
XLM 3  Not Updated
XLM 22
XMG 23
XMY 24
BLK 25
BURST 26
BYC 27
CANN 28
DGB 29
DGB 4  Not Updated
DOGE 30
EFL 31
EXCL 32
LTC 33
LTC 34
LTC 5  Not Updated
NAV 6  Not Updated
NAV 35
NEOS 36
NLG 37
NXT 38
POT 39
PTC 40
SLR 41
SPR 42
TRUST 43
VIA 7  Not Updated
VIA 44
VTC 45
XMR 8  Not Updated
XMR 46
XRP 47
XRP 48
XST 49
XCP 50
BITB 51
GEO 52
FLDC 53
GRC 54
FLO 55
DASH 56
DASH 57
DASH 9  Not Updated
CLAM 58
NBT 59
XEM 10  Not Updated
XEM 60
DMD 61
MUE 62
GAM 63
SPHR 64
OK 65
PPC 66
ERC 67
AEON 68
ETH 69
ETH 70
ETH 11  Not Updated
TX 71
BCY 72
EXP 73
OMNI 74
AMP 75
AUR 76
BNT 77
BNT 78


C:\Users\Administrator\Anaconda3\lib\site-packages\pyti\stochastic.py:21: RuntimeWarning: invalid value encountered in double_scalars
  np.min(data[idx+1-period:idx+1]))) for idx in range(period-1, len(data))]


RVR 79
EMC 80
FCT 81
EGC 82
SLS 83
GRS 12  Not Updated
GRS 84
RADS 85
DCR 86
BSD 87
XVG 88
XVG 89
MEME 90
GAME 91
STEEM 13  Not Updated
STEEM 92
2GIVE 93
LSK 14  Not Updated
LSK 94
BRK 95
WAVES 15  Not Updated
WAVES 96
LBC 97
SBD 98
BRX 99
ETC 100
ETC 101
ETC 16  Not Updated
STRAT 102
STRAT 103
COVAL 104
GLD 105
SYNX 106
UNB 107
EBST 108
VRM 109
NXS 17  Not Updated
NXS 110
REP 18  Not Updated
REP 111
SHIFT 112
ARDR 113
XZC 19  Not Updated
XZC 114
NEO 115
NEO 116
NEO 20  Not Updated
ZEC 21  Not Updated
ZEC 117
ZCL 118
IOP 119
GOLOS 120
UBQ 121
GBG 122
KMD 22  Not Updated
KMD 123
PIVX 23  Not Updated
PIVX 124
ION 125
LMC 126
SIB 127
QWARK 128
CRW 129
SWT 130
MLN 131
SEQ 132
ARK 133
ARK 134
DYN 135
TKS 136
MUSIC 137
DTB 138
INCNT 139
GBYTE 140
GNT 24  Not Updated
GNT 141
NXC 142
EDG 143
RLC 25  Not Updated
RLC 144
WINGS 26  Not Updated
WINGS 145
GUP 146
GNO 147
LUN 27  Not Updated
LUN 148
HMQ 149
ANT 150
SC 28  Not Updated
SC 151
BAT 29  Not Updated
BAT 152
ZEN 30  Not Updated
ZEN 153
QRL

In [ ]:
update_indicator(data_directory+'all_coins_day_full_1day_Cryptopia.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_3days_Cryptopia.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_7days_Cryptopia.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_10days_Cryptopia.csv',250,'10day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_14days_Cryptopia.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_min_full_1min.csv',250,'1min')
print("Done")

In [50]:
def resample(csv_filename,period,resampling_multiplier,exchange,datetimeformat_string,output_file_name):
    df_csv = pd.read_csv(csv_filename,dayfirst=True)
    #df_csv['time'] = df_csv.unix_timestamp.apply(lambda t: datetime.datetime.fromtimestamp(int(t)).strftime(datetimeformat_string))
    df_csv.unix_timestamp = pd.to_datetime(df_csv.unix_timestamp,unit="s",utc=True)
    #df_csv['time'].to_csv('time.csv')
    df_csv = df_csv.reset_index()
    #print(df_csv.columns)
    for indicator in indicator_list:
        if indicator not in df_csv.columns:
            df_csv[indicator] = np.nan
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    all_dataframes = []
    resampling_period = ""+str(resampling_multiplier)+frequency_resampling_period_mapping[period]
    output_csv_filename = output_file_name
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            #print(type(req_data.index))
            req_data = req_data.resample(resampling_period,level=2,closed='right',label='right').agg({'open': 'first', 
                                     'high': 'max', 
                                     'low': 'min', 
                                     'close': 'last',
                                    'volumeto':'sum',
                                        'volumefrom':'sum'})

            req_data['coin'] = coin_name
            req_data['exchange'] = key
            
            #print(req_data)
            req_data = req_data.reset_index()
            req_data['unix_timestamp'] =  req_data.unix_timestamp.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t), '%Y-%m-%d %H:%M:%S').timetuple()))
            #req_data['time'].to_csv("Req_data_Time.csv")
            req_data['time'] = pd.to_datetime(req_data.unix_timestamp,unit="s",utc=True)

            req_data = req_data.set_index(['coin','exchange','unix_timestamp'])
            i = i+1
            print(coin_name,i)
            #req_data = req_data.drop(labels=['time'],axis=1)
            all_dataframes.append(req_data)
    dataframe_answer = pd.concat(all_dataframes)
    #dataframe_answer['time'].to_csv('Time.csv')
    dataframe_answer.to_csv(output_csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [61]:
resample(data_csv_period_mapping['1day'],'day',3,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_3days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
BSD 99
XVG 100
XVG 101
MEME 102
GAME 103
STEEM 104
STEEM 105
2GIVE 106
LSK 107
LSK 108
BRK 109
WAVES 110
WAVES 111
LBC 112
SBD 113
BRX 114
ETC 115
ETC 116
ETC 117
STRAT 118
STRAT 119
COVAL 120
GLD 121
SYNX 122
UNB 123
EBST 124
VRM 125
NXS 126
NXS 127
REP 128
REP 129
SHIFT 130
ARDR 131
XZC 132


In [60]:
resample(data_csv_period_mapping['1day'],'day',7,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_7days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
BSD 99
XVG 100
XVG 101
MEME 102
GAME 103
STEEM 104
STEEM 105
2GIVE 106
LSK 107
LSK 108
BRK 109
WAVES 110
WAVES 111
LBC 112
SBD 113
BRX 114
ETC 115
ETC 116
ETC 117
STRAT 118
STRAT 119
COVAL 120
GLD 121
SYNX 122
UNB 123
EBST 124
VRM 125
NXS 126
NXS 127
REP 128
REP 129
SHIFT 130
ARDR 131
XZC 132


In [59]:
resample(data_csv_period_mapping['1day'],'day',10,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_10days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
BSD 99
XVG 100
XVG 101
MEME 102
GAME 103
STEEM 104
STEEM 105
2GIVE 106
LSK 107
LSK 108
BRK 109
WAVES 110
WAVES 111
LBC 112
SBD 113
BRX 114
ETC 115
ETC 116
ETC 117
STRAT 118
STRAT 119
COVAL 120
GLD 121
SYNX 122
UNB 123
EBST 124
VRM 125
NXS 126
NXS 127
REP 128
REP 129
SHIFT 130
ARDR 131
XZC 132


In [58]:
resample(data_csv_period_mapping['1day'],'day',14,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_14days.csv')

XWC 1
XDN 2
EMC2 3
THC 4
VTR 5
CURE 6
ENRG 7
KORE 8
PINK 9
RBY 10
RDD 11
ABY 12
BAY 13
BLOCK 14
CLOAK 15
CLOAK 16
DOPE 17
FTC 18
IOC 19
MONA 20
SYS 21
SYS 22
VRC 23
XLM 24
XLM 25
XMG 26
XMY 27
BLK 28
BURST 29
BYC 30
CANN 31
DGB 32
DGB 33
DOGE 34
EFL 35
EXCL 36
LTC 37
LTC 38
LTC 39
NAV 40
NAV 41
NEOS 42
NLG 43
NXT 44
POT 45
PTC 46
SLR 47
SPR 48
TRUST 49
VIA 50
VIA 51
VTC 52
XMR 53
XMR 54
XRP 55
XRP 56
XST 57
XCP 58
BITB 59
GEO 60
FLDC 61
GRC 62
FLO 63
DASH 64
DASH 65
DASH 66
CLAM 67
NBT 68
XEM 69
XEM 70
DMD 71
MUE 72
GAM 73
SPHR 74
OK 75
PPC 76
ERC 77
AEON 78
ETH 79
ETH 80
ETH 81
TX 82
BCY 83
EXP 84
OMNI 85
AMP 86
AUR 87
BNT 88
BNT 89
RVR 90
EMC 91
FCT 92
EGC 93
SLS 94
GRS 95
GRS 96
RADS 97
DCR 98
BSD 99
XVG 100
XVG 101
MEME 102
GAME 103
STEEM 104
STEEM 105
2GIVE 106
LSK 107
LSK 108
BRK 109
WAVES 110
WAVES 111
LBC 112
SBD 113
BRX 114
ETC 115
ETC 116
ETC 117
STRAT 118
STRAT 119
COVAL 120
GLD 121
SYNX 122
UNB 123
EBST 124
VRM 125
NXS 126
NXS 127
REP 128
REP 129
SHIFT 130
ARDR 131
XZC 132


In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',3,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_3days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',7,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_7days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',10,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_10days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',14,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_14days_Cryptopia.csv')

In [78]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',4,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_4hours.csv')

BTC 1
Done


In [79]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',6,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_6hours.csv')

BTC 1
Done


In [80]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',12,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_12hours.csv')

BTC 1
Done


In [81]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',3,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_3days.csv')

BTC 1
Done


In [82]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',7,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_7days.csv')

BTC 1
Done


In [83]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',14,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_14days.csv')

BTC 1
Done


In [51]:
def fetch_data_api(coin=COIN, to_curr=CURR, nperiods=1, period='1day',exchange_name=EXCHANGES[0]):
    """ Fetch data for coin over nperiods
        e.g. Get data for 'BTC' for past 12 hours in hours granularity
    """
    period = period.lower()
    func = function_period_mapping[period]
    if exchange_name == 'IDEX':
        to_curr = 'ETH'
    if coin == 'BTC':
        to_curr = 'USD'
    coin_last_nperiods = func(
        coin=coin,
        to_curr=to_curr,
        limit=nperiods,
        exchange=exchange_name
    )
    #print(coin_last_nperiods.index.get_level_values(0))
    #answer = coin_last_nperiods.iloc[-int(nperiods):]
    #print(answer.index.get_level_values(0))
    if coin_last_nperiods is not None:
        return coin_last_nperiods.iloc[-int(nperiods):]
    else:
        return None

In [52]:
def delete_latest_period_data(csv_filename,datetimeformat_string):
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M'))
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    final_dataframe = []
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data = req_data[:-1]
            #print(req_data.tail(1))
            final_dataframe.append(req_data)
    answer = pd.concat(final_dataframe).reset_index()
    #answer.to_csv('answer.csv')
    return answer

In [53]:
def update_csv_to_latest(period='1day',datetimeformat_string='%d-%m-%Y %H:%M',final_csv_column_order=[]):
    """ Update the csv for given period upto current time for coin """
    period = period.lower()
    csv_filename = data_csv_period_mapping[period]  # Get corr csv
    #csv_filename = 'Experiment.csv'
    #df_coin_period = pd.read_csv(csv_filename)  # , index_col=['coin', 'exchange']
    df_coin_period = delete_latest_period_data(csv_filename,datetimeStringformat_to_csv)
    #print(df_coin_period.tail(1)['time'])
    print("Updated Dataframe")
    csv_column_order = df_coin_period.columns.tolist()
    df_coin_period = df_coin_period.set_index(keys=['coin', 'exchange'])
    #df_coin_period.time = df_coin_period.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M'))
    
    lst_new_data = []
    PRINT_MSG = "{:15} {!s:20} {!s:>20} {:>10}"
    logging.info(PRINT_MSG.format("Exchange", "Last Updated Time", "Elapsed Time", "nPeriodsAgo"))
    coins_in_csv = list(df_coin_period.index.get_level_values(0).unique())
    #coins_in_csv = ['ZRX']
    i=0
    j=0
    for coin in coins_in_csv:
        df_coin_period_coin = df_coin_period.loc[coin]
        # Group by exchange, sort on timestamp, and get the last row of that particular coin
        last_update = df_coin_period_coin.groupby('exchange', group_keys=False).apply(lambda c: c.sort_values(by='unix_timestamp').tail(1))
        logging.info("-" * 10 + " For coin - {}".format(coin))
        
        for exchange in last_update.index.values: #For every coin exchange combination
            last_updated_time = int(last_update.loc[exchange]['unix_timestamp']) #Get the time of the last row
            #print(type(last_updated_time))
            try:
                # elapsed_time = datetime.datetime.now() - datetime.datetime.strptime(last_updated_time, '%Y-%m-%d %H:%M:%S')
                # elapsed_time = datetime.datetime.now() - datetime.datetime.strptime(last_updated_time, '%d-%m-%Y %H:%M')
                elapsed_time = int(time.time()) - last_updated_time
            except ValueError as e:
                logging.info("Failed to parse time {} for {}--{}".format(last_updated_time, coin, exchange))
                elapsed_time = datetime.datetime.now() - dateutil.parser.parse(last_updated_time)
            nperiods_ago = 0
            if period == '1day' or period == '1daycryptopia' or period =='1daybtcbitfinex':
                nperiods_ago = elapsed_time/(60*60*24)
            elif period == '1hour' or period == '1hourbtcbitfinex':
                 nperiods_ago = elapsed_time/(60*60)
            """
            nperiods_ago = elapsed_time / datetime.timedelta(days=1 if period == '1day' or period == '1daycryptopia' or period =='1daybtcbitfinex' else 0,
                                                             hours=1 if period == '1hour' or period == '1hourbtcbitfinex' else 0,
                                                             minutes=1 if period == '1min' else 0,
                                                             seconds=1)
            """
            nperiods_ago = np.floor(nperiods_ago)
            #print(nperiods_ago)
            logging.info(PRINT_MSG.format(exchange, last_updated_time, elapsed_time, nperiods_ago))
            #if coin == 'BTC':
              #  nperiods_ago = nperiods_ago-1
            if nperiods_ago > 0:
                
                logging.info("Updating data for {coin}-{exchange} from {last_updated_time}".format(
                    coin=coin, exchange=exchange, last_updated_time=last_updated_time)
                )
                #sys.exit("Testing")
                new_data_coin_period = fetch_data_api(
                    coin=coin,
                    nperiods=nperiods_ago,
                    period=period,
                    exchange_name=exchange
                )
                #print(new_data_coin_period.shape)
                if new_data_coin_period is None:
                    print(coin,exchange," Info Not available from API",str(j))
                    j =j+1
                    continue
                new_data_coin_period['coin'] = coin
                new_data_coin_period['exchange'] = exchange
                new_data_coin_period = new_data_coin_period.reset_index()
                new_data_coin_period['time'] =  pd.to_datetime(new_data_coin_period.unix_timestamp,unit="s",utc=True)
                i = i + 1
                #print(coin,exchange,i)
                lst_new_data.append(new_data_coin_period)
    
    #print("CSV column order "+str(csv_column_order))
    if lst_new_data:
        df_new_data = pd.concat(lst_new_data)
        df_new_data = df_new_data.reset_index()
        curr_columns = df_new_data.columns.tolist()
        #print("Current columns "+str(curr_columns))
        df_coin_period = df_coin_period.reset_index()
        csv_column_order = df_coin_period.columns.tolist()
        column_order = [col for col in csv_column_order if col in curr_columns]
        #print("New Column Order "+str(column_order))
        df_new_data = df_new_data.reindex(columns=column_order)
        df_coin_period=df_coin_period.append(df_new_data)
        #df_new_data.to_csv(csv_filename, mode='a', header=False,index=False,date_format="%d-%m-%Y %H:%M:%S")
        df_coin_period = df_coin_period[final_csv_column_order]
        df_coin_period.drop_duplicates(subset=['coin','exchange','unix_timestamp'],inplace=True)
        df_coin_period.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
        #df_new_data.to_csv('Put it into CSV.csv')
    print("Done")    

In [57]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'BBANDS_BANDWIDTH_PERCENT','MACD','MACD_SIGNAL','MACD_TEST','MONEY_FLOW_INDEX','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST','RSI','RSI_OVER_BOUGHT',
                 'RSI_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST','STOCHRSI_D','STOCHRSI_K','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','STOCH_PERCENT_D','STOCH_PERCENT_D_MONEY_FLOW_INDEX',
                 'STOCH_PERCENT_K','STOCH_PERCENT_K_MONEY_FLOW_INDEX']

update_csv_to_latest('1day',datetimeStringformat_to_csv,columns_order)

2018-07-23 10:51:56,437 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo
2018-07-23 10:51:56,474 - INFO - ---------- For coin - XWC
2018-07-23 10:51:56,496 - INFO - Bittrex         1532217600                         105716        1.0
2018-07-23 10:51:56,498 - INFO - Updating data for XWC-Bittrex from 1532217600


Updated Dataframe


2018-07-23 10:51:58,782 - INFO - ---------- For coin - XDN
2018-07-23 10:51:58,785 - INFO - Bittrex         1532217600                         105718        1.0
2018-07-23 10:51:58,788 - INFO - Updating data for XDN-Bittrex from 1532217600
2018-07-23 10:52:00,369 - INFO - ---------- For coin - EMC2
2018-07-23 10:52:00,370 - INFO - Bittrex         1532217600                         105720        1.0
2018-07-23 10:52:00,371 - INFO - Updating data for EMC2-Bittrex from 1532217600
2018-07-23 10:52:01,930 - INFO - ---------- For coin - THC
2018-07-23 10:52:01,931 - INFO - Bittrex         1532217600                         105721        1.0
2018-07-23 10:52:01,932 - INFO - Updating data for THC-Bittrex from 1532217600
2018-07-23 10:52:03,425 - INFO - ---------- For coin - VTR
2018-07-23 10:52:03,426 - INFO - Bittrex         1532217600                         105723        1.0
2018-07-23 10:52:03,427 - INFO - Updating data for VTR-Bittrex from 1532217600
2018-07-23 10:52:04,719 - INFO - -----

BSD Bittrex  Info Not available from API 0


2018-07-23 10:55:23,669 - INFO - Bittrex         1532217600                         105923        1.0
2018-07-23 10:55:23,670 - INFO - Updating data for XVG-Bittrex from 1532217600
2018-07-23 10:55:26,527 - INFO - ---------- For coin - MEME
2018-07-23 10:55:26,529 - INFO - Bittrex         1532217600                         105926        1.0
2018-07-23 10:55:26,530 - INFO - Updating data for MEME-Bittrex from 1532217600
2018-07-23 10:55:28,809 - INFO - ---------- For coin - GAME
2018-07-23 10:55:28,811 - INFO - Bittrex         1532217600                         105928        1.0
2018-07-23 10:55:28,812 - INFO - Updating data for GAME-Bittrex from 1532217600
2018-07-23 10:55:30,172 - INFO - ---------- For coin - STEEM
2018-07-23 10:55:30,174 - INFO - Binance         1532217600                         105930        1.0
2018-07-23 10:55:30,175 - INFO - Updating data for STEEM-Binance from 1532217600
2018-07-23 10:55:31,502 - INFO - Bittrex         1532217600                         105931 

SBD Bittrex  Info Not available from API 1


2018-07-23 10:55:49,350 - INFO - ---------- For coin - ETC
2018-07-23 10:55:49,352 - INFO - Binance         1532217600                         105949        1.0
2018-07-23 10:55:49,353 - INFO - Updating data for ETC-Binance from 1532217600
2018-07-23 10:55:51,309 - INFO - Bittrex         1532217600                         105951        1.0
2018-07-23 10:55:51,310 - INFO - Updating data for ETC-Bittrex from 1532217600
2018-07-23 10:55:54,021 - INFO - Kucoin          1532217600                         105954        1.0
2018-07-23 10:55:54,022 - INFO - Updating data for ETC-Kucoin from 1532217600
2018-07-23 10:55:55,346 - INFO - ---------- For coin - STRAT
2018-07-23 10:55:55,347 - INFO - Binance         1532217600                         105955        1.0
2018-07-23 10:55:55,348 - INFO - Updating data for STRAT-Binance from 1532217600
2018-07-23 10:55:56,969 - INFO - Bittrex         1532217600                         105956        1.0
2018-07-23 10:55:56,970 - INFO - Updating data for ST

Done


In [ ]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'BBANDS_BANDWIDTH_PERCENT','MACD','MACD_SIGNAL','MACD_TEST','MONEY_FLOW_INDEX','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST','RSI','RSI_OVER_BOUGHT',
                 'RSI_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST','STOCHRSI_D','STOCHRSI_K','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','STOCH_PERCENT_D','STOCH_PERCENT_D_MONEY_FLOW_INDEX',
                 'STOCH_PERCENT_K','STOCH_PERCENT_K_MONEY_FLOW_INDEX']
update_csv_to_latest('1dayCryptopia',datetimeStringformat_to_csv,columns_order)

In [66]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
update_csv_to_latest('1daybtcbitfinex',datetimeStringformat_to_csv,columns_order)

Updated Dataframe


2018-07-23 14:02:46,000 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo
2018-07-23 14:02:46,381 - INFO - ---------- For coin - BTC
2018-07-23 14:02:46,488 - INFO - Bitfinex        1531785600                         549166        6.0
2018-07-23 14:02:46,489 - INFO - Updating data for BTC-Bitfinex from 1531785600


Done


In [67]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
update_csv_to_latest('1hourbtcbitfinex',datetimeStringformat_to_csv,columns_order)

2018-07-23 14:02:53,810 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo
2018-07-23 14:02:53,816 - INFO - ---------- For coin - BTC
2018-07-23 14:02:53,817 - INFO - Bitfinex        1531890000                         444773      123.0
2018-07-23 14:02:53,819 - INFO - Updating data for BTC-Bitfinex from 1531890000


Updated Dataframe
Done


In [68]:
indicator_list_btc = ['unix_timestamp','UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']

In [69]:
def update_indicator_BTC(csv_filename,periods,timeframe,datetimeformat_string):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M:%S'))
    
    for indicator in indicator_list_btc:
        if indicator not in df_csv.columns and indicator not in df_csv.index:
            df_csv[indicator] = np.nan
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data2 = req_data.iloc[-periods:]

            start_date = req_data2.index.get_level_values(2)[0]
            end_date = req_data2.index.get_level_values(2)[req_data2.shape[0]-1]
            req_data2 = req_data[(req_data.index.get_level_values(2) >= start_date) & (req_data.index.get_level_values(2) <= end_date)]
            #print(req_data2)
            np_volumeto = np.array(req_data2.volumeto.values,dtype='f8')
            if len(np_volumeto)<20:
                j = j+1
                print(coin_name,j," Not Updated")
                continue
            req_data2['UPPER_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.upper_bollinger_band(req_data2.close.values,20)
            req_data2['MIDDLE_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.middle_bollinger_band(req_data2.close.values,20)
            req_data2['LOWER_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.lower_bollinger_band(req_data2.close.values,20)
            df_csv.update(req_data2)
            i = i+1
            print(coin_name,i)
            #print(df_csv.query('coin == @coin_name').tail(1))
            #sys.exit("Testing")
    df_csv.to_csv(csv_filename,date_format=datetimeformat_string)
    print("Done")

In [91]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

BTC 1
Done
Done


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [90]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_3days.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")  

BTC 1
Done
Done


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [89]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_7days.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")

BTC 1
Done
Done


In [88]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_day_full_14days.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")

BTC 1
Done
Done


In [87]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_1hour.csv',250,'1hour',datetimeStringformat_to_csv)
print("Done")

BTC 1
Done
Done


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [86]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_4hours.csv',250,'4hour',datetimeStringformat_to_csv)
print("Done")

BTC 1
Done
Done


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [85]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_6hours.csv',250,'6hour',datetimeStringformat_to_csv)
print("Done")

BTC 1
Done
Done


In [84]:
update_indicator_BTC(data_directory+'BTC_Bitfinex_hour_full_12hours.csv',250,'12hour',datetimeStringformat_to_csv)
print("Done")

BTC 1
Done
Done


In [142]:
def create_Histogram(csv_filename,periods,timeframe,datetimeformat_string):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    
    #print(closing_prices[closing_prices.size-1])
    first_value,last_value = df_csv['close'].min(),df_csv['close'].max()
    print(first_value,last_value)
    val = first_value + 200
    list_of_val = [first_value]
    while val <= last_value:
        list_of_val.append(val)
        list_of_val.append(val)
        val = val + 200
    list_of_val.append(val)
    list_of_val.append(val)
    it = iter(list_of_val)
    #print(list_of_val)
    z = zip(it,it)
    print(list(z))

In [143]:
create_Histogram(data_directory+'BTC_Bitfinex_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")

212.63 19210.0
[(212.63, 412.63), (412.63, 612.63), (612.63, 812.63), (812.63, 1012.63), (1012.63, 1212.6300000000001), (1212.6300000000001, 1412.6300000000001), (1412.6300000000001, 1612.6300000000001), (1612.6300000000001, 1812.6300000000001), (1812.6300000000001, 2012.6300000000001), (2012.6300000000001, 2212.6300000000001), (2212.6300000000001, 2412.6300000000001), (2412.6300000000001, 2612.6300000000001), (2612.6300000000001, 2812.6300000000001), (2812.6300000000001, 3012.6300000000001), (3012.6300000000001, 3212.6300000000001), (3212.6300000000001, 3412.6300000000001), (3412.6300000000001, 3612.6300000000001), (3612.6300000000001, 3812.6300000000001), (3812.6300000000001, 4012.6300000000001), (4012.6300000000001, 4212.6300000000001), (4212.6300000000001, 4412.6300000000001), (4412.6300000000001, 4612.6300000000001), (4612.6300000000001, 4812.6300000000001), (4812.6300000000001, 5012.6300000000001), (5012.6300000000001, 5212.6300000000001), (5212.6300000000001, 5412.6300000000001)

In [ ]:
scheduler = BackgroundScheduler()
scheduler.start()
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
scheduler.add_job(update_csv_to_latest, "cron", ['1daybtcbitfinex',datetimeStringformat_to_csv,columns_order], minute='*/1')


In [ ]:
scheduler.shutdown(wait=False)

In [ ]:
def changeCSVDateTimeFormat(csv_filename):
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True,infer_datetime_format =True)
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv['time'] = pd.to_datetime(df_csv.unix_timestamp,unit="s",utc=True)
    df_csv.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [ ]:
changeCSVDateTimeFormat('all_coins_day_full_1day.csv')

In [ ]:
changeCSVDateTimeFormat('all_coins_day_full_1day_Cryptopia.csv')

In [ ]:
changeCSVDateTimeFormat('BTC_Bitfinex_day_full_1day.csv')

In [ ]:
changeCSVDateTimeFormat('BTC_Bitfinex_hour_full_1hour.csv')

In [ ]:
print( type(datetime.datetime.now()))